In [1]:
# Install necessary libraries (if not installed)
!pip install transformers torch sentencepiece rouge-score nltk scikit-learn tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5404eb2fa53546007e4474417414d8d431fbfd00269c1cb4eb9694d536208267
  

In [2]:
# Install necessary libraries (if not installed)
# !pip install transformers torch sentencepiece rouge-score nltk scikit-learn tqdm

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm
from google.colab import files

# Upload CSV File
print("Upload your dataset (CSV file)...")
uploaded = files.upload()

# Get the uploaded file name
file_name = list(uploaded.keys())[0]
print(f"Dataset uploaded: {file_name}")

# Load dataset
df = pd.read_csv(file_name)

# Ensure the dataset contains the expected column
if 'sentence' not in df.columns:
    raise ValueError("Dataset must contain a column named 'sentence'")

# Check if CUDA is available (for GPU acceleration)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load DeepSeek-R1-Distill-Qwen-7B model
model_name = "deepseek-ai/deepseek-r1-distill-qwen-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

def paraphrase(text):
    """Paraphrase input text using DeepSeek-R1-Distill-Qwen-7B."""
    prompt = f"Paraphrase this sentence: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,  # Limit new tokens for paraphrasing
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            repetition_penalty=2.0
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply paraphrasing to the dataset
tqdm.pandas()
df["paraphrased_sentence"] = df["sentence"].progress_apply(paraphrase)

# Compute similarity using TF-IDF + Cosine Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["sentence"].tolist() + df["paraphrased_sentence"].tolist())

# Extracting the similarity scores
original_vectors = tfidf_matrix[:len(df)]
paraphrased_vectors = tfidf_matrix[len(df):]

similarities = [cosine_similarity(original_vectors[i], paraphrased_vectors[i])[0][0] for i in range(len(df))]

# Add similarity scores to DataFrame
df["similarity_score"] = similarities

# Calculate mean and standard deviation of similarity scores
mean_similarity = np.mean(similarities)
std_dev_similarity = np.std(similarities)

# Initialize ROUGE and BLEU score computation
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
bleu_scores = []
rouge_scores = {"rouge-1": [], "rouge-2": [], "rouge-l": []}

smoothing = SmoothingFunction().method1

for i in range(len(df)):
    ref = df.loc[i, "sentence"]
    pred = df.loc[i, "paraphrased_sentence"]

    # Compute BLEU score
    bleu = sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing)
    bleu_scores.append(bleu)

    # Compute ROUGE scores
    rouge_score = scorer.score(ref, pred)
    rouge_scores["rouge-1"].append(rouge_score["rouge1"].fmeasure)
    rouge_scores["rouge-2"].append(rouge_score["rouge2"].fmeasure)
    rouge_scores["rouge-l"].append(rouge_score["rougeL"].fmeasure)

# Add BLEU and ROUGE scores to DataFrame
df["bleu_score"] = bleu_scores
df["rouge-1"] = rouge_scores["rouge-1"]
df["rouge-2"] = rouge_scores["rouge-2"]
df["rouge-l"] = rouge_scores["rouge-l"]

# Calculate mean and standard deviation for BLEU and ROUGE scores
mean_bleu = np.mean(bleu_scores)
std_bleu = np.std(bleu_scores)

mean_rouge_1 = np.mean(rouge_scores["rouge-1"])
std_rouge_1 = np.std(rouge_scores["rouge-1"])

mean_rouge_2 = np.mean(rouge_scores["rouge-2"])
std_rouge_2 = np.std(rouge_scores["rouge-2"])

mean_rouge_l = np.mean(rouge_scores["rouge-l"])
std_rouge_l = np.std(rouge_scores["rouge-l"])

# Save results
output_file = "PP-002.csv"
df.to_csv(output_file, index=False)

# Provide download link for output file
files.download(output_file)

# Print evaluation results
print(f"Results saved to {output_file}")
print(f"Mean Similarity Score: {mean_similarity:.4f}")
print(f"Standard Deviation of Similarity Scores: {std_dev_similarity:.4f}")
print(f"Mean BLEU Score: {mean_bleu:.4f}, Standard Deviation: {std_bleu:.4f}")
print(f"Mean ROUGE-1 Score: {mean_rouge_1:.4f}, Standard Deviation: {std_rouge_1:.4f}")
print(f"Mean ROUGE-2 Score: {mean_rouge_2:.4f}, Standard Deviation: {std_rouge_2:.4f}")
print(f"Mean ROUGE-L Score: {mean_rouge_l:.4f}, Standard Deviation: {std_rouge_l:.4f}")


Upload your dataset (CSV file)...


Saving paraphtrase_dataset.csv to paraphtrase_dataset.csv
Dataset uploaded: paraphtrase_dataset.csv
Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

  6%|▋         | 30/480 [00:56<14:14,  1.90s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
 18%|█▊        | 88/480 [02:43<12:24,  1.90s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
 21%|██        | 99/480 [03:02<11:54,  1.87s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
 29%|██▉       | 139/480 [04:15<10:02,  1.77s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
 32%|███▏      | 155/480 [04:43<10:12,  1.88s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
 45%|████▌     | 216/480 [06

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Results saved to PP-002.csv
Mean Similarity Score: 0.4843
Standard Deviation of Similarity Scores: 0.1369
Mean BLEU Score: 0.1450, Standard Deviation: 0.0938
Mean ROUGE-1 Score: 0.3002, Standard Deviation: 0.0976
Mean ROUGE-2 Score: 0.2758, Standard Deviation: 0.0993
Mean ROUGE-L Score: 0.3002, Standard Deviation: 0.0976
